In [2]:
from env import host, user, password
from env import get_db_url
import pandas as pd
import numpy as np
url = f'mysql+pymysql://{user}:{password}@{host}/employees'


In [4]:
pd.read_sql('SELECT * FROM employees LIMIT 5 OFFSET 50', url)


,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10051,1953-07-28,Hidefumi,Caine,M,1992-10-15
1,10052,1961-02-26,Heping,Nitsch,M,1988-05-21
2,10053,1954-09-13,Sanjiv,Zschoche,F,1986-02-04
3,10054,1957-04-04,Mayumi,Schueller,M,1995-03-13
4,10055,1956-06-06,Georgy,Dredge,M,1992-04-27


Create a function named get_db_url. It should accept a username, hostname, password, and database name and return a url connection string formatted like in the example at the start of this lesson.

In [5]:
#this is done in env.py and imported above

In [3]:
test_url = get_db_url('employees')

In [4]:
pd.read_sql('SELECT * FROM employees LIMIT 5 OFFSET 50', test_url)

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10051,1953-07-28,Hidefumi,Caine,M,1992-10-15
1,10052,1961-02-26,Heping,Nitsch,M,1988-05-21
2,10053,1954-09-13,Sanjiv,Zschoche,F,1986-02-04
3,10054,1957-04-04,Mayumi,Schueller,M,1995-03-13
4,10055,1956-06-06,Georgy,Dredge,M,1992-04-27


a. Intentionally make a typo in the database url. What kind of error message do you see?

In [5]:
test_url = get_db_url('employeeasdass') # employees spelled wrong intentionally
# pd.read_sql('SELECT * FROM employees LIMIT 5 OFFSET 50', test_url)

# OperationalError: (pymysql.err.OperationalError) (1044, "Access denied for user 'innis_1662'@'%' to database 'employeeasdass'")
# (Background on this error at: https://sqlalche.me/e/14/e3q8)

b. Intentionally make an error in your SQL query. What does the error message look like?

In [6]:
test_url = get_db_url('employees')
#pd.read_sql('SELECTTTTTTT * FROM employees LIMIT 5 OFFSET 50', test_url) # select intentionally wrong
# error is ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'SELECTTTTTTT * FROM employees LIMIT 5 OFFSET 50' at line 1")
# [SQL: SELECTTTTTTT * FROM employees LIMIT 5 OFFSET 50]
# (Background on this error at: https://sqlalche.me/e/14/f405)

Read the employees and titles tables into two separate DataFrames.



In [8]:
employees = pd.read_sql('SELECT * FROM employees', url)
titles = pd.read_sql('SELECT * FROM titles', url)

How many rows and columns do you have in each DataFrame? Is that what you expected?



In [22]:
employees.shape

(300024, 6)

In [23]:
titles.shape

(443308, 4)

Display the summary statistics for each DataFrame.




In [24]:
employees.describe()

,emp_no
count,300024.000000
mean,253321.763392
std,161828.235540
min,10001.000000
25%,85006.750000
50%,249987.500000
75%,424993.250000
max,499999.000000


In [25]:
titles.describe()

,emp_no
count,443308.000000
mean,253075.034430
std,161853.292613
min,10001.000000
25%,84855.750000
50%,249847.500000
75%,424891.250000
max,499999.000000


How many unique titles are in the titles DataFrame?



In [12]:
titles.value_counts('title').count()


7

In [11]:
titles['title'].unique().size

7

What is the oldest date in the to_date column?



In [35]:
titles.sort_values(by='to_date')['to_date'].head(1)



16064    1985-03-01
Name: to_date, dtype: object

What is the most recent date in the to_date column?



In [57]:
new_titles = pd.read_sql('SELECT * FROM titles WHERE to_date != "9999-01-01"', url)
new_titles.sort_values(by='to_date')['to_date'].tail(1)

167151    2002-08-01
Name: to_date, dtype: object